In [ ]:
import json
from kreis_loader import get_kreise

kreise = get_kreise(returnGeometry=True)

with open("../kreise.json", "w") as json_file:
    json.dump(kreise, json_file)

In [ ]:
from data_handler import SQLite
import json

# Open the file for reading
with open('../../kreise.json', 'r') as json_file:
    kreise = json.load(json_file)

for kreis in kreise:
    kreis["attributes"]['KREISID'] = kreis["attributes"].pop('OBJECTID')

# kreis table
columns = {
        'KREISID': 'INTEGER PRIMARY KEY NOT NULL',
        'ags': 'TEXT',
        'gen': 'TEXT',
        'bez': 'TEXT',
        'ibz': 'INTEGER',
        'bem': 'TEXT',
        'sn_l': 'TEXT',
        'sn_r': 'TEXT',
        'sn_k': 'TEXT',
        'sn_v1': 'TEXT',
        'sn_v2': 'TEXT',
        'sn_g': 'TEXT',
        'fk_s3': 'TEXT',
        'nuts': 'TEXT',
        'wsk': 'TEXT', 
        'ewz': 'INTEGER',
        'kfl': 'REAL',
        'Shape__Area': 'REAL',
        'Shape__Length': 'REAL'
    }
with SQLite('ChargeApp.db') as db:
        db.create_table("kreis_table", columns)

# geometry table
columns = {
            'KREISID': 'INTEGER PRIMARY KEY NOT NULL',
            'GeoData': 'BLOB',
        }   
reference_key = {
        'table': 'kreis_table',
        'column': 'KREISID',
        'reference_column': 'KREISID'
    }
with SQLite('ChargeApp.db') as db:
    db.create_sub_table("geometry", columns, reference_key)

# station table
columns = {
    'OBJECTID': 'INTEGER PRIMARY KEY NOT NULL',
    'KREISID': 'INTEGER',
    'Betreiber': 'TEXT',
    'Straße': 'TEXT',
    'Hausnummer': 'TEXT',
    'Adresszusatz': 'TEXT',
    'Postleitzahl': 'INTEGER',
    'Ort': 'TEXT',
    'Bundesland': 'TEXT',
    'Kreis_kreisfreie_Stadt': 'TEXT',
    'Breitengrad': 'REAL',
    'Längengrad': 'REAL',
    'Inbetriebnahmedatum': 'TEXT',
    'Anschlussleistung': 'REAL',
    'Art_der_Ladeeinrichung': 'TEXT',
    'Anzahl_Ladepunkte': 'INTEGER',
    'Steckertypen1': 'TEXT',
    'P1__kW_': 'INTEGER',
    'Public_Key1': 'TEXT',
    'Steckertypen2': 'TEXT',
    'P2__kW_': 'REAL',
    'Public_Key2': 'TEXT',
    'Steckertypen3': 'TEXT',
    'P3__kW_': 'INTEGER',
    'Public_Key3': 'TEXT',
    'Steckertypen4': 'TEXT',
    'P4__kW_': 'INTEGER',
    'Public_Key4': 'TEXT'
}
reference_key = {
        'table': 'kreis_table',
        'column': 'KREISID',
        'reference_column': 'KREISID'
    }
with SQLite('ChargeApp.db') as db:
    db.create_sub_table("stations", columns, reference_key)
    

In [1]:
from kreis_loader import get_envelope
from stations_loader import stations_find, filter_stations
from data_handler import SQLite
import json

# Open the file for reading
with open('../../kreise.json', 'r') as json_file:
    kreise = json.load(json_file)

for kreis in kreise:
    kreis["attributes"]['KREISID'] = kreis["attributes"].pop('OBJECTID')


for kreis in kreise:
    kreis_id = kreis["attributes"].get('KREISID')

    ## Add Kreise
    with SQLite('ChargeApp.db') as db:
        db.insert_data(
            table_name="kreis_table",
            key_column="KREISID",
            data=kreis["attributes"]
        )

    polygon = kreis.get("geometry", None)

    ## Add geometry
    if polygon is not None:
        with SQLite('ChargeApp.db') as db:
            db.insert_data(
                table_name="geometry",
                key_column="KREISID",
                data={'GeoData': json.dumps(polygon)},
                key_value=kreis_id,
                reference_key={
                    'table': 'kreis_table',
                    'column': 'KREISID',
                    'reference_column': 'KREISID'
                }
            )
    
    # Compute the envelope and add it to the kreis
        envelope = get_envelope(polygon)
        with SQLite('ChargeApp.db') as db:
            db.add_column("kreis_table", "envelope", "TEXT")
            db.insert_data(
                table_name="kreis_table",
                key_column="KREISID",
                data={"envelope": kreis["envelope"]},
                key_value=kreis_id
            )


    

Data inserted or updated in table kreis_table.
Data inserted or updated in table geometry.
Column envelope already exists in table kreis_table.
Data inserted or updated in table kreis_table.
Data inserted or updated in table kreis_table.
Data inserted or updated in table geometry.
Column envelope already exists in table kreis_table.
Data inserted or updated in table kreis_table.
Data inserted or updated in table kreis_table.
Data inserted or updated in table geometry.
Column envelope already exists in table kreis_table.
Data inserted or updated in table kreis_table.
Data inserted or updated in table kreis_table.
Data inserted or updated in table geometry.
Column envelope already exists in table kreis_table.
Data inserted or updated in table kreis_table.
Data inserted or updated in table kreis_table.
Data inserted or updated in table geometry.
Column envelope already exists in table kreis_table.
Data inserted or updated in table kreis_table.
Data inserted or updated in table kreis_table

In [2]:
class StationsFinder:
    """
    Class for station finding
    """
    def __init__(self, base_url):
        self.base_url = base_url
        self.default_params = {
            "where": "1=1",
            "objectIds": "",
            "outFields": "*",
            "returnGeometry": "false",
            "returnIdsOnly": "false",
            "returnCountOnly": "false",
            "inSR": "4326",
            "outSR": "4326",
            "f": "json"
        }

        self.allowed_out_fields = [
            '*',
            'Betreiber',
            'Straße',
            'Hausnummer',
            'Ort',
            'Bundesland',
            'Kreis_kreisfreie_Stadt',
            'Breitengrad',
            'Längengrad',
            'Inbetriebnahmedatum',
            'Anschlussleistung',
            'Art_der_Ladeeinrichung',
            'Anzahl_Ladepunkte',
            'Steckertypen1',
            'P1__kW_',
            'Public_Key1',
            'Steckertypen2',
            'P2__kW_',
            'Public_Key2',
            'Steckertypen3',
            'P3__kW_',
            'Public_Key3',
            'Steckertypen4',
            'P4__kW_',
            'Public_Key4'
        ]

    def fetch_data(self, object_ids=None, **kwargs):
        """
        Fetch data from API
        """
        if object_ids:
            object_ids = ",".join(map(str, object_ids))

        params = self.default_params.copy()
        params['objectIds'] = object_ids
        params.update(kwargs)
        print(params)

        try:
            response = requests.get(self.base_url, params=params, timeout=10.0)
            response.raise_for_status()

            data_json = json.loads(response.text)
            if "error" in data_json:
                print(data_json)
                return None

            features = data_json.get("features", [])

            formatted_data = [
                {
                    **feature.get("attributes", {}),
                    "geometry": feature.get("geometry", {})
                }
                for feature in features
            ]

            return formatted_data

        except RequestException as error:
            print(f"An error occurred: {error}")
            return None

def stations_find(object_ids=None, **kwargs):
    """
    Function for retrieving stations
    """
    base_url = (
    'https://services2.arcgis.com/jUpNdisbWqRpMo35/arcgis/'
    'rest/services/Ladesaeulen_in_Deutschland/FeatureServer/0/query'
)
    api= StationsFinder(base_url)
    return api.fetch_data(object_ids, **kwargs)


envelope = '{12.2077887934618, 50.8096154655284, 12.6541150894765, 51.1037825036282}'
stations = stations_find(geometry = envelope)

{'attributes': {'ags': '01001', 'gen': 'Flensburg', 'bez': 'Kreisfreie Stadt', 'ibz': 40, 'bem': '--', 'sn_l': '01', 'sn_r': '0', 'sn_k': '01', 'sn_v1': '00', 'sn_v2': '00', 'sn_g': '000', 'fk_s3': 'R', 'nuts': 'DEF01', 'wsk': -2209161600000, 'ewz': 89934, 'kfl': 56.73, 'Shape__Area': 147888382.547852, 'Shape__Length': 74401.8518365568, 'KREISID': 1}, 'geometry': {'rings': [[[9.41266410790402, 54.8226409068342], [9.41318187527423, 54.8212363961779], [9.41337448934251, 54.8212465250665], [9.41481157011659, 54.8213220887413], [9.41607586892895, 54.8217187424077], [9.41793477789919, 54.8219589044247], [9.41913422912781, 54.8224626662259], [9.41999230617541, 54.8229455722613], [9.42119887745096, 54.8231711773981], [9.42293404888071, 54.8232229781175], [9.42333027149801, 54.8226205943784], [9.42347820246576, 54.8224915464673], [9.42417131109793, 54.8218868917314], [9.42432468226304, 54.8217813972641], [9.42490787842694, 54.8213802441682], [9.42531361420313, 54.8209467811392], [9.42559018751

In [7]:
envelope = '{12.2077887934618, 50.8096154655284, 12.6541150894765, 51.1037825036282}'
stations_find(geometry = envelope)

{'where': '1=1', 'objectIds': None, 'outFields': '*', 'returnGeometry': 'false', 'returnIdsOnly': 'false', 'returnCountOnly': 'false', 'inSR': '4326', 'outSR': '4326', 'f': 'json', 'geometry': '{12.2077887934618, 50.8096154655284, 12.6541150894765, 51.1037825036282}'}
An error occurred: HTTPSConnectionPool(host='services2.arcgis.com', port=443): Max retries exceeded with url: /jUpNdisbWqRpMo35/arcgis/rest/services/Ladesaeulen_in_Deutschland/FeatureServer/0/query?where=1%3D1&outFields=%2A&returnGeometry=false&returnIdsOnly=false&returnCountOnly=false&inSR=4326&outSR=4326&f=json&geometry=%7B12.2077887934618%2C+50.8096154655284%2C+12.6541150894765%2C+51.1037825036282%7D (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0xffff61f09060>: Failed to resolve 'services2.arcgis.com' ([Errno -2] Name or service not known)"))


In [ ]:
for kreis in kreise:
    ## Add stations
    stations = stations_find(geometry = envelope)
    stations_filtered = filter_stations(polygon, stations)
    for station in stations_filtered:
        station['KREISID'] = kreis_id

    with SQLite('ChargeApp.db') as db:
        db.insert_data(
            table_name="stations",
            key_column="OBJECTID",
            data=stations_filtered,
            reference_key={
                'table': 'kreis_table',
                'column': 'OBJECTID',
                'reference_column': 'KREISID'
            },
            strict=False
        )

In [ ]:
from kreis_loader import get_envelope
from stations_loader import stations_find, filter_stations
from data_handler import SQLite
import json

for kreis in kreise:
    kreis_id = kreis["attributes"].get('KREISID')

    ## Add Kreise
    with SQLite('ChargeApp.db') as db:
        db.insert_data(
            table_name="kreis_table",
            key_column="KREISID",
            data=kreis["attributes"]
        )

    polygon = kreis.get("geometry", None)

    ## Add geometry
    if polygon is not None:
        with SQLite('ChargeApp.db') as db:
            db.insert_data(
                table_name="geometry",
                key_column="KREISID",
                data={'GeoData': json.dumps(polygon)},
                key_value=kreis_id,
                reference_key={
                    'table': 'kreis_table',
                    'column': 'KREISID',
                    'reference_column': 'KREISID'
                }
            )

        # Compute the envelope and add it to the kreis
        envelope = get_envelope(polygon)
        with SQLite('ChargeApp.db') as db:
            db.add_column("kreis_table", "envelope", "TEXT")
            db.insert_data(
                table_name="kreis_table",
                key_column="KREISID",
                data={"envelope": kreis["envelope"]},
                key_value=kreis_id
            )

    ## Add stations
    stations = stations_find(geometry = envelope)
    stations_filtered = filter_stations(polygon, stations)
    for station in stations_filtered:
        station['KREISID'] = kreis_id

    with SQLite('ChargeApp.db') as db:
        db.insert_data(
            table_name="stations",
            key_column="OBJECTID",
            data=stations_filtered,
            reference_key={
                'table': 'kreis_table',
                'column': 'OBJECTID',
                'reference_column': 'KREISID'
            },
            strict=False
        )

In [ ]:
from data_handler import SQLiteFetcher
from map_drawer import DrawMap as dm

with SQLiteFetcher('../../ChargeApp.db', kreisid=1) as fetcher:
    flensburg = fetcher.fetch_geometry_data("geometry")
    envelope = fetcher.fetch_rows('kreis_table')[0]["envelope"]
    stations = fetcher.fetch_rows('stations')

# Create an instance of DrawMap
dm = dm()
station_map = dm.plot_regions(flensburg)
station_map = dm.add_envelope(envelope)
station_map = dm.add_stations(stations)
station_map